In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [84.8 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,733 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [3]:
 from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [4]:
# Read in CSV
from pyspark import SparkFiles
df = spark.read.csv(SparkFiles.get("/content/dataset.csv"),sep=",", escape='"', encoding="utf-8", quote='"',  header=True)
df.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|reuters highlight...|    0|
|washington reuter...|    0|
|moscow reuters ru...|    0|
|st century wire s...|    1|
|mitt romney top a...|    1|
+--------------------+-----+
only showing top 5 rows



In [23]:
from pyspark.sql.types import IntegerType
df = df.withColumn("label", df["label"].cast(IntegerType()))

In [24]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
df = df.withColumn('length', length(df['text']))
df.show(3)

+--------------------+-----+------+
|                text|label|length|
+--------------------+-----+------+
|reuters highlight...|    0|  1902|
|washington reuter...|    0|  2767|
|moscow reuters ru...|    0|   884|
+--------------------+-----+------+
only showing top 3 rows



In [25]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [26]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [27]:
 # Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, idf, clean_up])

In [28]:
 # Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df)
cleaned = cleaner.transform(df)

In [29]:
 # Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(262145,[1872,225...|
|    0|(262145,[1512,274...|
|    0|(262145,[1022,538...|
|    1|(262145,[619,958,...|
|    1|(262145,[1512,352...|
|    1|(262145,[2437,293...|
|    0|(262145,[1097,141...|
|    0|(262145,[723,1968...|
|    1|(262145,[1765,230...|
|    0|(262145,[1797,383...|
|    1|(262145,[5381,153...|
|    1|(262145,[2306,780...|
|    1|(262145,[2325,311...|
|    0|(262145,[1772,314...|
|    1|(262145,[531,1004...|
|    0|(262145,[2325,107...|
|    0|(262145,[25328,27...|
|    1|(262145,[619,3774...|
|    1|(262145,[2437,313...|
|    1|(262145,[1512,271...|
+-----+--------------------+
only showing top 20 rows



In [30]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [31]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(5)

+--------------------+-----+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|label|length|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------------------+-----+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|aboard seefuchs r...|    0|  2206|[aboard, seefuchs...|[aboard, seefuchs...|(262144,[1968,230...|(262144,[1968,230...|(262145,[1968,230...|[-13458.991280253...|[1.0,2.0704741858...|       0.0|
|absent parent cas...|    1|  1134|[absent, parent, ...|[absent, parent, ...|(262144,[991,1880...|(262144,[991,1880...|(262145,[991,1880...|[-6907.9103970071...|[2.25455277928886...|       1.0|
|absolutely doubt ...|    1|  

In [32]:
 # Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.914352
